In [4]:
from scipy.io import loadmat
import mat73
import matplotlib.pyplot as plt
from hmmlearn import hmm
import numpy as np
from tqdm import notebook
from scipy import stats
from scipy.signal import hilbert
from scipy.signal import butter, lfilter
from scipy import signal
from scipy.fft import fftshift
from kneed import KneeLocator

In [5]:
def bandpower(data, sf, band, method='welch', window_sec=None, relative=False):
    """Compute the average power of the signal x in a specific frequency band.

    Requires MNE-Python >= 0.14.

    Parameters
    ----------
    data : 1d-array
      Input signal in the time-domain.
    sf : float
      Sampling frequency of the data.
    band : list
      Lower and upper frequencies of the band of interest.
    method : string
      Periodogram method: 'welch' or 'multitaper'
    window_sec : float
      Length of each window in seconds. Useful only if method == 'welch'.
      If None, window_sec = (1 / min(band)) * 2.
    relative : boolean
      If True, return the relative power (= divided by the total power of the signal).
      If False (default), return the absolute power.

    Return
    ------
    bp : float
      Absolute or relative band power.
    """
    from scipy.signal import welch
    from scipy.integrate import simps
    from mne.time_frequency import psd_array_multitaper

    band = np.asarray(band)
    low, high = band

    # Compute the modified periodogram (Welch)
    if method == 'welch':
        if window_sec is not None:
            nperseg = window_sec * sf
        else:
            nperseg = (2 / low) * sf

        freqs, psd = welch(data, sf, nperseg=nperseg)

    elif method == 'multitaper':
        psd, freqs = psd_array_multitaper(data, sf, adaptive=True,
                                          normalization='full', 
                                          verbose=0)

    # Frequency resolution
    freq_res = freqs[1] - freqs[0]

    # Find index of band in frequency vector
    idx_band = np.logical_and(freqs >= low, freqs <= high)

    # Integral approximation of the spectrum using parabola (Simpson's rule)
    bp = simps(psd[idx_band], dx=freq_res)

    if relative:
        bp /= simps(psd, dx=freq_res)
    return psd, freqs, idx_band, bp

In [6]:
filepath = '../processed_data/lfp.mat'
ey_lfp = mat73.loadmat(filepath)
fs = int(ey_lfp["lfp"]["samplerate"][0])
print(f'sampling frequency is {fs} Hz')
full_lfp = np.vstack([ey_lfp['lfp']['data'][ch] for ch in range(len(ey_lfp['lfp']['data']))]).T
print(f'full LFP lasted for {full_lfp.shape[0]/(fs*60*60)} hours across {full_lfp.shape[1]} channels')

sampling frequency is 1250 Hz
full LFP lasted for 4.540507555555555 hours across 39 channels


In [7]:
# define frequency ranges for band powers
freq_bands = [[3, 8]]
bands = ['theta']
# initialize list
band_pow = []

# define window in seconds for frequency analysis
window_dur = 1
# define time total in hours for analysis
duration_total = 1

win_size = int(window_dur*fs) # sampling at 1250 Hz, 1s is 1250 samples

# win_num = int(full_lfp.shape[0]/win_size) # how many seconds total
nun_overlaps = 5
win_num = int(duration_total*60*60/window_dur*nun_overlaps) #  hours 

# for each band
for i, freq_band in enumerate(notebook.tqdm(freq_bands)):
    instant_band_pow_all = []
    # for each channel
    for ch in notebook.tqdm(range(1)):
        instant_band_pow_single = []
        # for each analysis window
        for win in range(win_num): 
            # window will be minimum nyquist 2/low_cut
            psd, freqs, idx_band, bp = bandpower(full_lfp[int(win*win_size*.2):int(win*win_size*.2)+win_size, ch], # int(win*win_size*1/nun_overlaps), int(win*win_size*1/nun_overlaps)+win_size
                                                 fs, 
                                                 freq_band, 
                                                #  win_sec=0.2,
                                                #  'multitaper',
                                                 relative=False,
                                                )
            # a list of instantaneous single band power single channel
            # instant_band_pow_single.append(np.mean(psd[idx_band])) 
            instant_band_pow_single.append(bp/(freq_band[1]-freq_band[0])) 
        # a list of 39 channels, single band power
        # instant_band_pow_all.append(np.hstack((instant_band_pow_single))) 
    # band_pow.append(instant_band_pow_all)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [12]:
arr = np.hstack((instant_band_pow_single))
arr.shape

(18000,)

In [11]:
arr = np.array(instant_band_pow_single)
arr.shape

(18000,)